In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pickle

In [2]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import datetime


In [8]:
class straight_route:
    def __init__(self):
        pass

    def print_solution(self):
        """Prints solution on console."""
        print(f'Objective: {self.solution.ObjectiveValue()}')
        total_time = 0
        total_load = 0

        for vehicle_id in range(self.data['num_vehicles']):
            self.straight_route = []
            index = self.routing.Start(vehicle_id)
            plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
            route_time = 0
            route_time_lst = [0]
            route_load = 0
            i=0

            while not self.routing.IsEnd(index):
                node_index = self.manager.IndexToNode(index)
                route_load += self.data['demands'][node_index]
            
                previous_index = index
                index = self.solution.Value(self.routing.NextVar(index))
                route_time_lst.append(self.routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id))
                route_time += self.routing.GetArcCostForVehicle(
                    previous_index, index, vehicle_id)

                plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(self.data['name_lst'][node_index], self.data['vehicle_capacities'][vehicle_id] - route_load, self.data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
                i+=1
                self.straight_route.append(node_index)

            node_index = self.manager.IndexToNode(index)
            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(self.data['name_lst'][node_index], self.data['vehicle_capacities'][vehicle_id] - route_load, self.data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            plan_output += '서비스 종료\n\n'
            plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
            plan_output += 'Load of the route: {}\n'.format(route_load)
            print(plan_output)

            total_time += route_time
            total_load += route_load

        print('\n')
        print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
        print('Total load of all routes: {}'.format(total_load))
        return self.straight_route
   

    def get_straight_route(self, time_matrix, input_node, name_lst, demand):
        """Entry point of the program."""
        self.data = {}
        self.data['name_lst'] = name_lst
        self.data['new_time_matrix'] = pd.DataFrame(time_matrix)[input_node].iloc[input_node].to_numpy()
        self.data['demands'] = demand
        self.data['vehicle_capacities'] = [5]
        self.data['num_vehicles'] = 1
        self.data['starts'] = [0]
        self.data['ends'] = [input_node[int(np.argmax(self.data['new_time_matrix'][0]))]]
        # Create the routing index manager.
        self.manager = pywrapcp.RoutingIndexManager(len(self.data['new_time_matrix']),
                                           self.data['num_vehicles'], self.data['starts'], self.data['ends'])

        # Create Routing Model.
        self.routing = pywrapcp.RoutingModel(self.manager)


        # 1-1. Define cost of each arc.
        def distance_callback(from_index, to_index):
            """Returns the manhattan distance between the two nodes."""
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            to_node = self.manager.IndexToNode(to_index)
            return self.data['new_time_matrix'][from_node][to_node]

        transit_callback_index = self.routing.RegisterTransitCallback(distance_callback)
        self.routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # 1-2.Add Distance constraint.
        dimension_name = 'Distance'
        self.routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            8000,  # vehicle maximum travel distance
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = self.routing.GetDimensionOrDie(dimension_name)
        distance_dimension.SetGlobalSpanCostCoefficient(100)

        # 2-1.Define cost of each node.
        def demand_callback(from_index):
            """Returns the demand of the node."""
            # Convert from routing variable Index to demands NodeIndex.
            from_node = self.manager.IndexToNode(from_index)
            return self.data['demands'][from_node]

        demand_callback_index = self.routing.RegisterUnaryTransitCallback(
            demand_callback)

        # 2-2.Add Capacity constraint.
        self.routing.AddDimensionWithVehicleCapacity(
            demand_callback_index,
            0,  # null capacity slack
            self.data['vehicle_capacities'],  # vehicle maximum capacities
            True,  # start cumul to zero
            'Capacity')

        # Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

        # Solve the problem.
        self.solution = self.routing.SolveWithParameters(search_parameters)

        # Print solution on console.
        if self.solution:
            self.print_solution()
            return self.straight_route
        else:
          print(self.solution)



In [9]:
time_matrix = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')
node = np.load('../locate_call/강남역_경로매트릭스_예시(분).npy')[:10,:10]
bus_name_lst = ['강남역','학동역','현대아파트','신당동.청구역','동묘앞','안암오거리','서초동삼성아파트','방배역','사당역13번출구','남성초등학교','언북중학교입구',
'세관앞','신구중학교','금옥초등학교.금호대우아파트','장충체육관앞','대광고등학교앞','서초역2번출구','청호나이스','방배동신동아아파트','서울고사거리','방배동래미안아파트','서울교통공사','구름다리','이수역','사당역']

In [10]:
([[0, 7, 8, 9, 6], [0, 3, 4, 5], [0, 2, 1]],
 [[0, 1, 1, 1, 1], [0, 1, 1, 1], [0, 2, 2]])

([[0, 7, 8, 9, 6], [0, 3, 4, 5], [0, 2, 1]],
 [[0, 1, 1, 1, 1], [0, 1, 1, 1], [0, 2, 2]])

In [11]:
pd.DataFrame(time_matrix)[[0, 3, 4, 5]].iloc[[0, 3, 4, 5]].to_numpy()[0]

array([   0,  994, 1420, 1558])

In [13]:
s = straight_route()
s.get_straight_route(time_matrix, [0, 3, 4, 5], bus_name_lst, [0, 1, 1, 1])

: 

: 